# 한영 번역 서비스 만들기  M2M100

In [1]:
# !pip install sentencepiece

In [3]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 한국어/영어 번역기 입니다. m2m100모델로 만들었어요."
en_text = "This is a Korean to English translator. I made it using the m2m100 model."

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# translate Hindi to French
tokenizer.src_lang = "ko"
encoded_ko = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
# => "La vie est comme une boîte de chocolat."
print(translated_text)

# translate Chinese to English
tokenizer.src_lang = "en"
encoded_en = tokenizer(en_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
translated_text2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
# => "Life is like a box of chocolate."
print(translated_text2)

This is a Korean/English translator. made with m2m100 model.
이것은 한국어에서 영어로 번역기입니다. m2m100 모델을 사용하여 만들었습니다.
